# Descarga de datos

In [2]:
from google.colab import output
!pip install gdown
!gdown https://drive.google.com/uc?id=1L5kNJEwRRlNtBvXVt4KZB0EiTQk23iaV # pokedex_processed.csv
!gdown https://drive.google.com/uc?id=19dw2vVgF6pht_-GF5_IvxrVLOSkT8kCQ # pokedex_processed.json
output.clear()

# Datos

Los datos corresponden al mundo de _Pokémon_ separado en dos bases de datos: Una relacional SQL (PostgreSQL) y otra de documentos NoSQL (MongoDB). Ambas son levantadas dentro del entorno de ejecución del notebook en Google Colab.

A continuación se crea y describe a cada una:

# PostgreSQL

Cada columna de la tabla de pokedex se describe a continuación:

```text
###################
## Tabla pokedex ##
###################

Id           : (int) Identificador único en el Pokédex
Name         : (str) Nombre del Pokémon
Category     : (str) La "especie" del Pokémon (en Pokémon Sol y Luna)
Height (m)   : (int) Altura en metros
Weight (kg)  : (int) Masa en kilogramos
Capture Rate : (int) Cuantifica la probabilidad de capturar al Pokémon (a mayor valor, más probable).
Egg Steps    : (int) Cuantifica el número de pasos que requiere en el juego para salir de un huevo.
Exp Group    : (str) Categoría que determina cuanta experiencia necesita para subir de nivel.
HP           : (int) Vida
Attack       : (int) Ataque
Defense      : (int) Defensa
Sp. Attack   : (int) Ataque Especial
Sp. Defense  : (int) Defensa Especial
Speed        : (int) Velocidad

```

In [3]:
# install
!apt update
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# pin package versions for google colab compatibility
!pip install SQLAlchemy==1.4.46
!pip install ipython-sql==0.4.1
# set connection
%load_ext sql
%config SqlMagic.feedback=False
%config SqlMagic.autopandas=True
%config SqlMagic.displaycon = False
%sql postgresql+psycopg2://@/postgres
output.clear()

In [4]:
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine("postgresql+psycopg2://@/postgres", echo=False)
df_pokedex = pd.read_csv("pokedex_processed.csv", index_col=False)
df_pokedex.to_sql("pokedex", con=engine, if_exists="replace", index=False)

809

In [5]:
%%sql
SELECT *
FROM pokedex
LIMIT 10

,Id,Name,Category,Height (m),Weight (kg),Capture Rate,Egg Steps,Exp Group,HP,Attack,Defense,Sp. Attack,Sp. Defense,Speed
0,1,Bulbasaur,Seed Pokémon,0.7,6.9,45,5120,Medium Slow,45,49,49,65,65,45
1,2,Ivysaur,Seed Pokémon,1.0,13.0,45,5120,Medium Slow,60,62,63,80,80,60
2,3,Venusaur,Seed Pokémon,2.0,100.0,45,5120,Medium Slow,80,82,83,100,100,80
3,4,Charmander,Lizard Pokémon,0.6,8.5,45,5120,Medium Slow,39,52,43,60,50,65
4,5,Charmeleon,Flame Pokémon,1.1,19.0,45,5120,Medium Slow,58,64,58,80,65,80
5,6,Charizard,Flame Pokémon,1.7,90.5,45,5120,Medium Slow,78,84,78,109,85,100
6,7,Squirtle,Tiny Turtle Pokémon,0.5,9.0,45,5120,Medium Slow,44,48,65,50,64,43
7,8,Wartortle,Turtle Pokémon,1.0,22.5,45,5120,Medium Slow,59,63,80,65,80,58
8,9,Blastoise,Shellfish Pokémon,1.6,85.5,45,5120,Medium Slow,79,83,100,85,105,78
9,10,Caterpie,Worm Pokémon,0.3,2.9,255,3840,Medium Fast,45,30,35,20,20,45


## MongoDB

La colección pokedex tiene documentos con 4 entradas: Un identificador único del pokémon y arreglos con información acerca del mismo. A continuación se detalla cada uno:

```text
#######################
## Colección pokedex ##
#######################

"pokemon_id": : (int)       Número identificador único en el Pokédex
"abilities"   : (List[str]) Lista de las habilidades del Pokémon
"moves"       : (List[str]) Lista de los movimientos del Pokémon
"types"       : (List[str]) Lista de los tipos del Pokémon
```

In [6]:
!apt install mongodb
!service mongodb start
!pip install pymongo
!mongoimport --db local --collection pokedex --jsonArray < 'pokedex_processed.json'
output.clear()

In [7]:
from pymongo import MongoClient
client = MongoClient()
client.list_database_names() # ['admin', 'local']
db = client['local']
collection = db['pokedex']

Para tener una idea de cómo se ven los datos en la colección, aquí les dejamos impresión del primer Pokemon en la colección:

In [8]:
import pprint
for pokemon in collection.find({}).limit(1):
    pprint.pprint(pokemon)

{'_id': ObjectId('648d3d0ed6e1e8dd70fdd03d'),
 'abilities': ['Overgrow', 'Chlorophyll'],
 'moves': ['Tackle is described as A physical attack in which the user charges '
           'and slams into the target with its whole body.',
           'Growl is described as The user growls in an endearing way, making '
           'opposing Pokémon less wary. This lowers their Attack stat.',
           'Vine Whip is described as The target is struck with slender, '
           'whiplike vines to inflict damage.',
           'Poison Powder is described as The user scatters a cloud of '
           'poisonous dust that poisons the target.',
           'Sleep Powder is described as The user scatters a big cloud of '
           'sleep-inducing dust around the target.',
           'Take Down is described as A reckless, full-body charge attack for '
           'slamming into the target. This also damages the user a little.',
           'Razor Leaf is described as Sharp-edged leaves are launched to '
    

# Preguntas

Para resolver estas preguntas será necesario que consultes a `PostgreSQL` y/o `MongoDB`.

En este `notebook` puedes almacenar el resultado de una consulta `PostgreSQL` en una variable como un `pandas.DataFrame` de la siguiente forma:

```python
df = %sql SELECT "Name" FROM pokedex WHERE "Id" = 99;
```

## Pregunta 1
Entregue el nombre (name) y la cantidad de habilidades (abilites) de los 10 pokemones más pesados en kilogramos

In [9]:
df = %sql SELECT "Name", "Id" FROM pokedex ORDER BY "Weight (kg)" DESC LIMIT 10;

pokemon_dict = df.to_dict(orient= 'records')
print('Nombre', 'Cantidad de habilidades')
for pokemon in pokemon_dict:
    pokemon_id = pokemon["Id"]
    nombre = pokemon["Name"]
    for pokemon in collection.find({"pokemon_id": pokemon_id}):
      habilidades= (pokemon["abilities"])
    print(nombre, len(habilidades))
    habilidades_count = collection.count_documents({'pokemon_id': pokemon_id, 'abilities': {'$exists': True}})
    pokemon['Habilidades'] = habilidades_count
#for pokemon in collection.find('pokemon_id' in df):
  #pprint.pprint(pokemon['abilities'])

Nombre Cantidad de habilidades
Cosmoem 1
Celesteela 1
Groudon 1
Mudsdale 3
Guzzlord 1
Stakataka 1
Melmetal 1
Giratina 3
Dialga 2
Metagross 2


## Pregunta 2

Nombre los 3 pokemones con más movimientos. Se recomienda revisar la documentación de MongoDB, particularmente de [`$sort`](https://www.mongodb.com/docs/manual/reference/operator/aggregation/sort/) y [`$limit`](https://www.mongodb.com/docs/manual/reference/operator/aggregation/limit/).

In [25]:
ids_vistos = set()
resultado = collection.find({}, {"pokemon_id": 1, "moves": 1}).sort([("moves", -1)]).limit(3)
lista_id = []
for pokemon in resultado:
    pokemon_id = pokemon["pokemon_id"]
    if pokemon_id not in ids_vistos:
      moves_count = len(pokemon["moves"])
      ids_vistos.add(pokemon_id)
      lista_id.append(pokemon_id)
query = """
SELECT "Name"
FROM pokedex
WHERE "Id" IN ({})
""".format(', '.join(str(id) for id in lista_id))

df = %sql $query
print(df)


         Name
0     Venonat
1    Venomoth
2  Togedemaru


## Pregunta 3

Cuál es el elemento de los pokemones más densos? Indique los 5 elementos de pokemones más densos. Asuma que la densidad es el peso dividido en la altura, y que los pokemones tienen un solo tipo (ignore todos los elementos de cada pokemon excepto uno).

ser muestran los tipos de los 5 pokemones más densos


In [11]:
query = """
SELECT "Name", "Id", "Weight (kg)" / "Height (m)" AS density
FROM pokedex
ORDER BY density DESC
LIMIT 5
"""
df = %sql $query
densidad_dict = df.to_dict(orient= 'records')

for pokemon in densidad_dict:
    id = pokemon['Id']
    resultado = collection.find_one({'pokemon_id': id})
    # Verificar si se encontró el Pokémon en la base de datos de MongoDB
    if resultado:
        # Obtener los atributos deseados del resultado de la consulta
        tipo = resultado['types']
        name = pokemon['Name']
        # Imprimir los datos del Pokémon
        print('Tipo:', tipo[0])


Tipo: Psychic
Tipo: Ground
Tipo: Steel
Tipo: Steel
Tipo: Ground


## Pregunta 4

Vea si existe alguna correlación entre algún atributo (ataque, defensa, ataque especial, defensa especial, velocidad y HP) y la cantidad de movimientos que tienen los pokemones.

Hay algun otro par de variables (de las mencionadas anteriormente) que tengan mayor correlación entre sí? Indique el par de atributos que tengan la mayor correlación entre sí y haga un gráfico de dispersión para visualizar la correlación.

In [41]:
df = %sql SELECT "Name", "Id", "HP" FROM pokedex;
ids_vistos = set()
resultado = collection.find({}, {"pokemon_id": 1, "moves": 1}).sort([("moves", -1)])

diccionario_movimientos = {}

for pokemon in resultado:
    pokemon_id = pokemon["pokemon_id"]
    if pokemon_id not in ids_vistos:
        moves_count = len(pokemon["moves"])
        ids_vistos.add(pokemon_id)
        diccionario_movimientos[pokemon_id] = moves_count

print(diccionario_movimientos)

# Crea una nueva columna en el DataFrame utilizando el diccionario de movimientos
df['Num_Movimientos'] = df['Id'].map(diccionario_movimientos)
# Reordena las columnas según tus preferencias
primeras_cinco_lineas = df.head(5)
print(primeras_cinco_lineas)
correlation = df[["HP", "Num_Movimientos"]].corr()
print(correlation)



{777: 15, 48: 14, 49: 18, 54: 16, 55: 18, 79: 16, 80: 17, 96: 16, 97: 18, 128: 14, 163: 18, 164: 19, 199: 18, 203: 17, 234: 15, 241: 13, 270: 13, 271: 13, 302: 20, 308: 21, 313: 16, 314: 16, 325: 15, 326: 17, 340: 19, 372: 14, 373: 18, 375: 14, 376: 15, 380: 19, 381: 19, 385: 15, 386: 14, 371: 13, 408: 12, 409: 13, 486: 14, 494: 17, 517: 17, 605: 18, 606: 19, 644: 15, 720: 19, 765: 15, 787: 18, 791: 17, 81: 18, 82: 20, 137: 15, 145: 16, 181: 21, 205: 20, 233: 16, 299: 16, 377: 14, 378: 14, 379: 16, 462: 23, 474: 17, 476: 21, 599: 15, 600: 15, 601: 17, 603: 15, 604: 10, 649: 20, 738: 15, 796: 18, 143: 17, 175: 15, 176: 17, 194: 13, 195: 13, 206: 21, 218: 16, 219: 17, 230: 13, 261: 16, 262: 21, 287: 11, 289: 14, 316: 16, 317: 18, 322: 15, 323: 16, 358: 15, 369: 14, 399: 14, 400: 17, 433: 8, 449: 11, 450: 14, 480: 12, 513: 16, 616: 15, 734: 16, 735: 16, 775: 13, 47: 13, 46: 12, 123: 18, 127: 16, 207: 15, 212: 18, 253: 16, 254: 18, 291: 15, 335: 16, 347: 15, 348: 14, 402: 15, 472: 18, 542:

el ataque tiene la mayor relacion con 0.253665

Attack  Num_Movimientos: 0.253665
Defense  Num_Movimientos: 0.212476
Sp. Attack Num_Movimientos: 0.181285
Sp. Defense Num_Movimientos: 0.226473
Speed Num_Movimientos:0.125093
HP  Num_Movimientos:0.18065

Los atributos con mayor correlación son La defensa y la defensa especial.

In [43]:
df = %sql SELECT "Attack",	"Defense",	"Sp. Attack",	"Sp. Defense",	"Speed", "HP" FROM pokedex;
correlation_matrix = df.corr()
correlation_pairs = correlation_matrix.unstack().sort_values(ascending=False)
print(correlation_pairs)


Attack       Attack         1.000000
Defense      Defense        1.000000
Speed        Speed          1.000000
Sp. Defense  Sp. Defense    1.000000
Sp. Attack   Sp. Attack     1.000000
HP           HP             1.000000
Defense      Sp. Defense    0.506163
Sp. Defense  Defense        0.506163
             Sp. Attack     0.481435
Sp. Attack   Sp. Defense    0.481435
             Speed          0.439785
Speed        Sp. Attack     0.439785
Defense      Attack         0.439335
Attack       Defense        0.439335
HP           Attack         0.437478
Attack       HP             0.437478
Sp. Attack   HP             0.377330
HP           Sp. Attack     0.377330
Sp. Defense  HP             0.366738
HP           Sp. Defense    0.366738
Attack       Speed          0.330985
Speed        Attack         0.330985
Attack       Sp. Attack     0.326032
Sp. Attack   Attack         0.326032
HP           Defense        0.237430
Defense      HP             0.237430
Sp. Defense  Speed          0.204384
S

## Importante:
Para las últimas preguntas será necesario que crees y consultes un índice de texto para el arreglo de strings `moves`.

In [12]:
# Aquí escribe el código para crear el índice de texto
collection.create_index([("moves", "text")])


'moves_text'

## Pregunta 5
Entregue el nombre de todos los pokemones los cuales en sus movimientos contengan las palabras `fire` y `thunder` y tengan un Ataque Especial (Sp. Attack) entre `110` y `130` (ambos valores incluidos)

In [13]:
result = collection.find({
    "$and": [
        {"moves": {"$regex": "fire", "$options": "i"}},
        {"moves": {"$regex": "thunder", "$options": "i"}}
    ]
})
lista_id = []
lista_movimiento =[]
for pokemon in result:
  lista_id.append(pokemon["pokemon_id"])
  lista_movimiento.append([pokemon["moves"]])
query = """
SELECT "Name"
FROM pokedex
WHERE "Sp. Attack" >= 110 AND "Sp. Attack" <= 130 AND "Id" IN ({})
""".format(", ".join(str(id) for id in lista_id))
df = %sql $query
print(df)


        Name
0   Magneton
1     Zapdos
2   Ampharos
3   Houndoom
4     Raikou
5  Salamence
6  Magnezone
7  Magmortar


## Pregunta 6
Entregue el nombre junto con la suma de las estadísticas de combate (HP, Attack, Defense, Sp. Attack, Sp. Defense y Speed) de los pokemones que en sus movimientos contengan las palabras `ice`, `freeze`, `snow` o `cold`, **pero no** las palabras `water` y `wind`. Deben estar ordenado alfabéticamente por su nombre.

In [20]:
resultado = collection.find({
    "$or": [
        {"moves": {"$regex": "ice", "$options": "i"}},
        {"moves": {"$regex": "freeze", "$options": "i"}},
        {"moves": {"$regex": "snow", "$options": "i"}},
        {"moves": {"$regex": "cold", "$options": "i"}}
    ]
})


lista_id = []
for pokemon in resultado:
  lista_id.append(pokemon["pokemon_id"])

resultado2 = collection.find({
    "$and": [
    {"moves": {"$regex": "water", "$options": "i"}},
    {"moves": {"$regex": "wind", "$options": "i"}}]})
lista_id2 = []
for pokemon in resultado2:
  lista_id2.append(pokemon["pokemon_id"])
contador = 0
i = 0
while i <= len(lista_id) - contador:
  i += 1
  if lista_id[i] in lista_id2:
    lista_id.pop(i)
    contador += 1
query = """
SELECT DISTINCT "Name", "HP" + "Attack" + "Defense" + "Sp. Attack" + "Sp. Defense" + "Speed" AS "Suma"
FROM pokedex
WHERE "Id" IN ({})
ORDER BY "Name" ASC
""".format(", ".join(str(id) for id in lista_id))

df = %sql $query
print(df)


           Name  Suma
0     Abomasnow   494
1    Aerodactyl   515
2         Aipom   360
3     Alomomola   470
4       Altaria   490
..          ...   ...
251      Zekrom   680
252     Zoroark   510
253       Zorua   330
254       Zubat   245
255    Zweilous   420

[256 rows x 2 columns]
